## Declare Models

Here, we define _module-level_ constructs that will __form the structures__ which we will be `querying` from the database. This structure, known as a `Declarative Mapping`, _defines at once_ both a __Python object model__, as well as __database metadata__ that _describes real SQL tables_ that `exist, or will exist`, in a particular database.

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import declarative_base, relationship

In [2]:
Base = declarative_base()

In [3]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [4]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

The _mapping_ starts with a `base class`, which above is called `Base`, and is created by calling upon the `declarative_base()` function, which produces a __new `base` class__.

_Individual mapped classes_ are then _created_ by making __subclasses of `Base`__. A _mapped class_ typically refers to a `single particular database table`, the name of which is indicated by using the __`__tablename__`__ _class-level attribute_.

Next, _columns_ that are part of the table are declared, by _adding attributes_ __linked__ to the `Column` construct. `Column` describes _all aspects_ of a `database column`, including __typing information__ with _type objects_ such as `Integer` and `String` as well as __server defaults__ and __constraint information__, such as _membership_ within the `primary key` and `foreign keys`.

_All_ `ORM mapped classes` __require__ _at least_ __one column__ be declared as part of the `primary key`, typically by using the `Column.primary_key` parameter on those `Column` objects that should be part of the key. In the above example, the `User.id` and `Address.id` columns are _marked_ as __primary key__.

Taken together, the _combination_ of a `string table name` as well as a `list of column declarations` is __referred__ towards in SQLAlchemy as __`table metadata`__. Setting up `table metadata` using both `Core` and `ORM` approaches is introduced in the `SQLAlchemy 1.4/2.0 Tutorial` at `Working with Database Metadata`. The above mapping is an example of what's referred towards as `Declarative Table configuration`.

Other _Declarative directives_ are available, most commonly the `relationship()` construct indicated above. In contrast to the _column-based attributes_, `relationship()` denotes a __linkage between two `ORM classes`__. In the above example, `User.addresses` __links `User` to `Address`__, and `Address.user` __links `Address` to `User`__. The `relationship()` construct is introduced in the `SQLAlchemy 1.4/2.0 Tutorial` at `Working with Related Objects`.

Finally, the above example classes include a `__repr__()` method, which is _not required_ but is __useful for debugging__.